In [12]:
from google.colab import files
uploaded = files.upload()  # pick your JSON file
CONFIG_PATH = next(iter(uploaded))  # use the uploaded filename
print("Using:", CONFIG_PATH)


Saving sim-config.json to sim-config.json
Using: sim-config.json


In [19]:
import json
from playwright.async_api import async_playwright

In [20]:
URL = "https://basaga.org/basaga_files/partificial-life/index.html"
CONFIG_PATH = "config.json"       # or the path you set earlier
STEPS_TO_WAIT = 10
TIMEOUT_MS = 30000

In [21]:
async def run_sim_async(url: str, config_path: str, steps_to_wait: int = 10, timeout_ms: int = 30000):
    cfg = json.load(open(config_path, "r", encoding="utf-8"))

    async with async_playwright() as p:
        browser = await p.chromium.launch(
            headless=True,
            args=["--no-sandbox", "--disable-dev-shm-usage"]
        )
        context = await browser.new_context()
        page = await context.new_page()

        # Optional: print page console logs (note: .text is a property)
        page.on("console", lambda msg: print("PAGE:", msg.text))

        await page.goto(url, wait_until="domcontentloaded")

        # Wait for the sim API to exist
        await page.wait_for_function(
            "() => window.simAPI && typeof window.simAPI.start === 'function'",
            timeout=timeout_ms
        )

        # Load config and start
        await page.evaluate("cfg => window.simAPI.loadConfig(cfg)", cfg)
        await page.evaluate("() => window.simAPI.reset()")
        await page.evaluate("() => window.simAPI.start()")

        # Wait until we have enough metric samples
        await page.evaluate(
            f"() => window.simAPI.waitUntil(m => m.step >= {steps_to_wait}, {{ timeoutMs: {timeout_ms} }})"
        )

        # Read latest metrics
        metrics = await page.evaluate("() => window.simAPI.getMetrics()")
        await browser.close()

    arr = [metrics['cluster'], metrics['entropy'], metrics['speed'], metrics['change'], metrics['cvi']]
    return arr, metrics

In [22]:
arr, full = await run_sim_async(URL, CONFIG_PATH, STEPS_TO_WAIT, TIMEOUT_MS)
print("Metrics array [cluster, entropy, speed, change, cvi]:", arr)
print("Full metrics object:", full)

PAGE: simAPI initialized successfully! Version: 1.0.0
Metrics array [cluster, entropy, speed, change, cvi]: [0.168, 13.714199251055483, 0.13907247202870596, 3.833039300104793, 0.1896520476702364]
Full metrics object: {'cluster': 0.168, 'entropy': 13.714199251055483, 'speed': 0.13907247202870596, 'change': 3.833039300104793, 'cvi': 0.1896520476702364, 'step': 10}
